# TMBD Movie Database Project
- Scotty Thomason

## MySQL database on Movies from a subset of IMDB's publicly available dataset

In [1]:
import pandas as pd
import numpy as np
import os,json

## Akas DataFrame

In [2]:
# akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
# akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
# akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

akas = pd.read_csv("Data/title_akas.csv.gz", low_memory = False)

In [3]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [4]:
akas.replace({'\\N':np.nan}, inplace = True)

In [5]:
us_mask = (akas['region'] == 'US')
filter_akas = akas.loc[us_mask]

In [6]:
filter_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [7]:
filter_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1408757 entries, 5 to 34697872
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1408757 non-null  object
 1   ordering         1408757 non-null  int64 
 2   title            1408757 non-null  object
 3   region           1408757 non-null  object
 4   language         3796 non-null     object
 5   types            970593 non-null   object
 6   attributes       45819 non-null    object
 7   isOriginalTitle  1407412 non-null  object
dtypes: int64(1), object(7)
memory usage: 96.7+ MB


In [8]:
## Save current dataframe to file.
# filter_akas.to_csv("Data/filtered_akas.csv.gz",compression='gzip',index=False)

## Basics DataFrame

In [9]:
#Data Sets
# basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [10]:
# Loading in the Data and saving it locally
# basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
# basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

In [11]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(filter_akas['titleId'])
keepers

0           True
1           True
2          False
3          False
4           True
           ...  
9559978    False
9559979    False
9559980    False
9559981    False
9559982    False
Name: tconst, Length: 9559983, dtype: bool

In [12]:
basics = basics[keepers]
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1324305 entries, 0 to 9559938
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   tconst          1324305 non-null  object
 1   titleType       1324305 non-null  object
 2   primaryTitle    1324305 non-null  object
 3   originalTitle   1324305 non-null  object
 4   isAdult         1324305 non-null  object
 5   startYear       1324305 non-null  object
 6   endYear         1324305 non-null  object
 7   runtimeMinutes  1324305 non-null  object
 8   genres          1324305 non-null  object
dtypes: object(9)
memory usage: 101.0+ MB


In [14]:
basics.replace({'\\N':np.nan}, inplace = True)

In [15]:
# Eliminate movies that are null for runtimeMinutes
basics.dropna(subset=['runtimeMinutes'], inplace = True)
basics.dropna(subset=['genres'], inplace = True)

In [16]:
basics['titleType'].value_counts()

short           271405
movie           196125
tvEpisode       169471
video           113550
tvSeries         35619
tvMovie          25620
tvSpecial         8915
tvMiniSeries      5529
tvShort           4082
videoGame          197
Name: titleType, dtype: int64

In [17]:
mask1 = (basics['titleType'] == 'movie')
filter_basics = basics.loc[mask1]

In [18]:
mask2 = (filter_basics['startYear'] >= '2000') & (filter_basics['startYear'] < '2022')
filter_basics1 = filter_basics.loc[mask2]

In [19]:
filter_basics1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [20]:
# Exclude movies that are included in the documentary category.
is_documentary = filter_basics1['genres'].str.contains('documentary',case=False)
filter_basics2 = filter_basics1[~is_documentary]

In [21]:
filter_basics2.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [22]:
filter_basics2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80484 entries, 34803 to 9559749
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          80484 non-null  object
 1   titleType       80484 non-null  object
 2   primaryTitle    80484 non-null  object
 3   originalTitle   80484 non-null  object
 4   isAdult         80484 non-null  object
 5   startYear       80484 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  80484 non-null  object
 8   genres          80484 non-null  object
dtypes: object(9)
memory usage: 6.1+ MB


In [23]:
## Save current dataframe to file.
# filter_basics2.to_csv("Data/filtered_basics.csv.gz",compression='gzip',index=False)

## Third DataFrame

In [24]:
# ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
# ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
# ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

In [25]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1948
1,tt0000002,5.8,264
2,tt0000003,6.5,1776
3,tt0000004,5.6,179
4,tt0000005,6.2,2581


In [26]:
ratings.replace({'\\N':np.nan}, inplace = True)

In [27]:
keepers = ratings['tconst'].isin(filter_akas['titleId'])
ratings = ratings[keepers]

In [28]:
## Save current dataframe to file.
# ratings.to_csv("Data/filtered_ratings.csv.gz",compression='gzip',index=False)

In [29]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486714 entries, 0 to 1274619
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         486714 non-null  object 
 1   averageRating  486714 non-null  float64
 2   numVotes       486714 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 14.9+ MB


# Part 2:  Extract from TMDB

- There is no financial information included in the IMDB data (e.g. budget or revenue). Analyze which movies are successful. The Movie Database (TMDB) as a great source of financial data
- Extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R) "Certification".

## Using tmdbsimple Package
- We will be using a Python package called tmdbsimple that will make it easier to extract the data we need without manually constructing the URLs for our API calls.

In [30]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [31]:
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'filtered_akas.csv.gz',
 'filtered_basics.csv.gz',
 'filtered_ratings.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_results_combined.csv.gz']

In [32]:
with open('/Users/scyjt/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [33]:
tmdb.API_KEY =  login['api-key']

In [34]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
                
        # Sets file's current position at offset.
        file.seek(0)
        
        # convert back to json.
        json.dump(file_data, file)

In [35]:
# define custom dunction
def get_movie_with_rating(movie_id):

    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    
    # Loop through countries in releases
    for c in releases['countries']:
        
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
        
    return info

In [36]:
new_basics = pd.read_csv("Data/filtered_basics.csv.gz", low_memory = False)

In [37]:
YEARS_TO_GET = [2000, 2001]

In [38]:
errors = [ ]

In [39]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

# If it does not exist: create it
if file_exists == False:
    
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
#Saving new year as the current df
df = new_basics.loc[new_basics['startYear']==YEAR].copy()

# saving movie ids to list
movie_ids = df['tconst'].copy()

In [41]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [42]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [43]:
#Get index and movie id from list
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
    try:
        # Retrieve the data for the movie id
        temp = get_movie_with_rating(movie_id)
            
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
            
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
            
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2001:   0%|          | 0/244 [00:00<?, ?it/s]

In [44]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [45]:
print(f"- Total errors: {len(errors)}")

- Total errors: 244


## Part 3 - Construct and export a MySQL database

In [55]:
basics_df = pd.read_csv('Data/filtered_basics.csv.gz', low_memory = False)
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


In [56]:
## create a col with a list of genres
basics_df['genres_split'] = basics_df['genres'].str.split(',')
basics_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,[Drama]
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama,[Drama]
...,...,...,...,...,...,...,...,...,...,...
80479,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019,NaN,74,Drama,[Drama]
80480,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy","[Comedy, Drama, Fantasy]"
80481,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama,[Drama]
80482,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller","[Action, Adventure, Thriller]"


In [57]:
exploded_genres = basics_df.explode('genres_split')
exploded_genres

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama,Drama
...,...,...,...,...,...,...,...,...,...,...
80482,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Action
80482,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Adventure
80482,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller",Thriller
80483,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History",Drama


In [58]:
unique_genres = sorted(exploded_genres['genres_split'].unique())

In [59]:
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


In [60]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

In [61]:
## make new integer genre_id and drop string genres
title_genres['genre_id'] = title_genres['genres_split'].map(genre_map)
title_genres = title_genres.drop(columns='genres_split')

In [63]:
genre_lookup = pd.DataFrame({'genre_name': genre_map.keys(), 'genre_id': genre_map.values()})
genre_lookup.head()

,genre_name,genre_id
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


In [62]:
basics_df.set_index('int_index').to_sql('table_name', engine, index=True)

KeyError: "None of ['int_index'] are in the columns"